<a href="https://colab.research.google.com/github/abigailibarrola/data301-figure-skating/blob/master/DATA_301_Final_Project_Data_Collection_%26_Cleaning_Abigail_Ibarrola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection and Cleaning


In [0]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup

# Data Collection: Scraping Web Data

I will be scraping data off of the website [skatingscores.com](https://skatingscores.com/), which contains pre-calculated total, TES, PCS, etc. score data for all judges in all major senior and junior level competitions beginning the 2010-2011 season. Calculating the individual judge scores manually requires a lot of time, especially with all the rule variations and value conversions changes every year.

I compiled a .csv containing the links to all of the competitions I will be using for my data. I selected all major international competitions for every season beginning from the 2010-2011 season, including Grand Prix events, Four Continents, European Champions, World Championships, and The Olympics.

In [64]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')
data_dir = "/content/gdrive/My Drive/DATA 301/"
df_comps = pd.read_csv(data_dir + "senior_skate_comps.csv")
df_comps

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,comp_id,link,date
0,WC_1011,https://skatingscores.com/1011/wc/,2011-04-25
1,4CC_1011,https://skatingscores.com/1011/4cc/,2011-02-15
2,EC_1011,https://skatingscores.com/1011/ec/,2011-01-24
3,GPF_1011,https://skatingscores.com/1011/gpf/,2010-12-08
4,GPFRA_1011,https://skatingscores.com/1011/gpfra/,2010-11-25
...,...,...,...
96,GPJPN_1920,https://skatingscores.com/1920/gpjpn/,2019-11-22
97,GPRUS_1920,https://skatingscores.com/1920/gprus/,2019-11-15
98,GPUSA_1920,https://skatingscores.com/1920/gpusa/,2019-10-18
99,GPCHN_1920,https://skatingscores.com/1920/gpchn/,2019-11-08


All of the scraped data will be stored in the corresponding array, which will then be converted into DataFrames.



In [0]:
all_data = {
    "MEN_SP" : [],
    "MEN_FS" : [],
    "LADIES_SP" : [],
    "LADIES_FS" : [],
    "SINGLES_JUMPS" : [],
    "SINGLES_SPINS" : [],
    "SINGLES_PCS" : [],
    "JUDGES_PANELS" : [],
    "JUDGES_INFO" : {}
}

## Functions for scraping web data

These are functions for scraping general skater information for each competition they perform. Because the data are in tables, it requires some maneuvering to get the desired cell.

In [0]:
def get_skater_info(containers, j_scores_container, comp_id, event):
  skater_info_container = containers[0]
  tes_scores_container = containers[3]
  pcs_scores_container = containers[7]
  skater_entry = {}

  # Place comp and event info in entry
  skater_entry["comp_id"] = comp_id
  skater_entry["event"] = event

  # Grab skater name
  name = skater_info_container.find("td", {"class" : "l tl"}).find_all("span")
  skater_fullname = name[0].text + " " + name[1].text
  skater_entry["skater_fullname"] = skater_fullname

  # Grab skater nationality & competition ranking
  nat_rank_container = skater_info_container.find_all("td", {"class" : "c tl"}, limit=2)
  nat = nat_rank_container[1].find("a")
  skater_entry["skater_nat"] = nat.text
  comp_rank = nat_rank_container[0].text
  skater_entry["comp_rank"] = comp_rank

  # Grab skater's final score & deductions
  final_score = skater_info_container.find_all("td", {"class" : "r"})
  skater_entry["total_score"] = final_score[1].text
  skater_entry["ded"] = final_score[0].text

  # Grab skater's program base value
  tes_scores = tes_scores_container.find_all("td", {"class" : "r eltot"})
  skater_entry["bv"] = tes_scores[0].contents[2]

  # Grab skater's final TES
  skater_entry["tes"] = tes_scores[1].contents[2]
  # Grab skater's final PCS
  skater_entry["pcs"] = pcs_scores_container.find("td", {"class" : "r pcs"}).contents[2]

  # Grab skater's J1-9 TES scores
  j_tes = j_scores_container[1].find_all("td", {"class" : "c"})
  j_tes = [s.text for s in j_tes]
  if len(j_tes) == 9:
    for i in range(0, 9):
      skater_entry["j%d_tes" % (i + 1)] = j_tes[i]
  else:
    for i in range(0, len(j_tes)):
      skater_entry["j%d_tes" % (i + 1)] = j_tes[i]
    for i in range(len(j_tes), 9):
      skater_entry["j%d_tes" % (i + 1)] = None

  # Grab skater's J1-9 PCS scores
  j_pcs = j_scores_container[2].find_all("td", {"class" : "c"})
  j_pcs = [s.text for s in j_pcs]
  if len(j_pcs) == 9:
    for i in range (0, 9):
      skater_entry["j%d_pcs" % (i + 1)] = j_pcs[i]
  else:
    for i in range(0, len(j_pcs)):
      skater_entry["j%d_pcs" % (i + 1)] = j_pcs[i]
    for i in range(len(j_pcs), 9):
      skater_entry["j%d_pcs" % (i + 1)] = None

  # Add entry to overall data
  all_data[event].append(skater_entry)

  # Return skater full name to be used in other tables
  return skater_fullname

In [0]:
def get_jumps_data(tes_container, comp_id, event, skater_fullname):
  jumps = tes_container.find_all("td", {"class" : "jump"})
  for jump in jumps:
    jump_entry = {}
    jump_entry["element"] = jump.text
    jump_entry["comp_id"] = comp_id
    jump_entry["skater_fullname"] = skater_fullname
    jump_entry["event"] = event
    jump_vals = jump.parent.find_all("td", {"class" : "r"}, limit=4)
    jump_entry["bv"] = jump_vals[2].text
    jump_entry["goe"] = jump_vals[3].text
    all_data["SINGLES_JUMPS"].append(jump_entry)

def get_spins_data(tes_container, comp_id, event, skater_fullname):
  spins = tes_container.find_all("td", {"class" : "spin"})
  for spin in spins:
    spin_entry = {}
    spin_entry["element"] = spin.text
    spin_entry["comp_id"] = comp_id
    spin_entry["skater_fullname"] = skater_fullname
    spin_entry["event"] = event
    spin_vals = spin.parent.find_all("td", {"class" : "r"}, limit=4)
    spin_entry["bv"] = spin_vals[2].text
    spin_entry["goe"] = spin_vals[3].text
    all_data["SINGLES_SPINS"].append(spin_entry)

def get_tes_data(containers, comp_id, event, skater_fullname):
  tes_container = containers[2]

  # Grab jumps element, bv, goe, mistakes
  get_jumps_data(tes_container, comp_id, event, skater_fullname)

  # Get spins data
  get_spins_data(tes_container, comp_id, event, skater_fullname)

def get_pcs_data(containers, comp_id, event, skater_fullname):
  pcs_container = containers[6]
  pcs_vals = pcs_container.find_all("td", {"class" : "r"})
  pcs_entry = {}
  pcs_entry["comp_id"] = comp_id
  pcs_entry["event"] = event
  pcs_entry["skater_fullname"] = skater_fullname
  pcs_entry["skating_skills"] = pcs_vals[1].contents[2]
  pcs_entry["transitions"] = pcs_vals[2].contents[2]
  pcs_entry["performance"] = pcs_vals[3].contents[2]
  pcs_entry["composition"] = pcs_vals[4].contents[2]
  pcs_entry["interpretation"] = pcs_vals[5].contents[2]

  all_data["SINGLES_PCS"].append(pcs_entry)

In [0]:
def get_data_singles(link, comp_id, event):
  response = requests.get(link)
  soup = BeautifulSoup(response.content, "html.parser")
  skater_tables = soup.find_all("div", {"class" : "skat-wrap"})

  for skater_table in skater_tables:
    table_containers = skater_table.find_all("table", {"class" : "ptab"})
    j_scores_container = skater_table.find_all("tr", {"class" : "s tally"})
    
    skater_fullname = get_skater_info(table_containers, j_scores_container, comp_id, event)
    get_tes_data(table_containers, comp_id, event, skater_fullname)
    get_pcs_data(table_containers, comp_id, event, skater_fullname)

In [0]:
def get_data_judges(link, comp_id, event1, event2):
  response = requests.get(link)
  soup = BeautifulSoup(response.content, "html.parser")
  judges_tables = soup.find_all("div", {"class" : "col-12 col-t-6 col-d-6"})

  event1_table = judges_tables[0]
  parse_judges_table(event1_table, comp_id, event1)
  if (comp_id != 'GPFRA_1516'): # The FS event for this competition was canceled
    event2_table = judges_tables[1]
    parse_judges_table(event2_table, comp_id, event2)

def parse_judges_table(table, comp_id, event):
  comp_panel_entry = {}
  comp_panel_entry["comp_id"] = comp_id
  comp_panel_entry["event"] = event
  span_idx = -1
  j_nats = table.find_all("a", {"class" : "fl"})
  j_names = table.find_all("span")

  for i in range(1, 10):
    j_fullname = j_names[i + span_idx].text + " " + j_names[i + (span_idx + 1)].text
    span_idx += 1
    j_nat = j_nats[i - 1]["title"]
    comp_panel_entry["j%d_name" % i] = j_fullname
    all_data["JUDGES_INFO"][j_fullname] = j_nat

  all_data["JUDGES_PANELS"].append(comp_panel_entry)

In [0]:
def get_events(comp_id, comp_link, event):
  if event == "MEN":
    event_link = "men"
  elif event == "LADIES":
    event_link = "ladies"

  sp_data_id = event + "_SP"
  fs_data_id = event + "_FS"

  # Process SP event
  sp_link = comp_link + event_link + "/short/"
  get_data_singles(sp_link, comp_id, sp_data_id)

  # Process FS event
  ladies_fs_link = comp_link + event_link + "/long/"
  if (comp_id != 'GPFRA_1516'): # The FS event for this competition was canceled
    get_data_singles(ladies_fs_link, comp_id, fs_data_id)

  # Process judges panel
  ladies_panel_link = comp_link + event_link + "/panel/"
  get_data_judges(ladies_panel_link, comp_id, sp_data_id, fs_data_id)

## Running through and collecting scraped data

In [9]:
# Iterate through all major senior competitions
i = 1
for row in df_comps.itertuples(index=True):
  # Print every fifth competition processed, for sanity check
  if (i % 5) == 0:
    print("Processing Competition #%d" % i)

  comp_id = getattr(row, "comp_id")
  comp_link = getattr(row, "link")

  # Get Men events data
  get_events(comp_id, comp_link, "MEN")

  # Get Ladies events data
  get_events(comp_id, comp_link, "LADIES")

  i += 1

Processing Competition #5
Processing Competition #10
Processing Competition #15
Processing Competition #20
Processing Competition #25
Processing Competition #30
Processing Competition #35
Processing Competition #40
Processing Competition #45
Processing Competition #50
Processing Competition #55
Processing Competition #60
Processing Competition #65
Processing Competition #70
Processing Competition #75
Processing Competition #80
Processing Competition #85
Processing Competition #90
Processing Competition #95
Processing Competition #100


## Converting to DataFrames

In [0]:
# Convert to DataFrames
df_men_sp = pd.DataFrame(all_data["MEN_SP"])
df_men_fs = pd.DataFrame(all_data["MEN_FS"])
df_ladies_sp = pd.DataFrame(all_data["LADIES_SP"])
df_ladies_fs = pd.DataFrame(all_data["LADIES_FS"])
df_jumps = pd.DataFrame(all_data["SINGLES_JUMPS"])
df_spins = pd.DataFrame(all_data["SINGLES_SPINS"])
df_pcs = pd.DataFrame(all_data["SINGLES_PCS"])
df_judge_panels = pd.DataFrame(all_data["JUDGES_PANELS"])
df_judges = pd.DataFrame(list(all_data["JUDGES_INFO"].items()), 
                         columns=["judge_fullname", "judge_nat"])

# Data Cleaning

### Cleaning up elements data

When an error is made on a particular element (i.e. a jump element), the error is indicated by a symbol next to the element with the error.

For example, 
*   A triple lutz jump is denoted by **3Lz**.
*   If a triple lutz is executed with the wrong edge, it receives an edge call, which is denoted on the score sheet with an "e". Thus, a triple lutz with an edge call is **3Lze**.
*   If a triple lutz is underrotated, it is denoted by a "<". Thus, a underrotated triple lutz is **3Lz<**.
*   If a triple lutz is downgraded, it is denoted by a "<<". Thus, a downgraded triple lutz is **3Lz<<**.
*   If a triple lutz is invalid, it is denoted by a "*". Thus, an invalid triple lutz is **3lz***.

I want to compare the same elements across all skaters, regardless of any errors. Thus, I strip off the characters and phrases that symbolize an error, and create cells based on what is found in the original string.









In [38]:
df_jumps.iloc[-5:]
# Example, jump #30943 is an underrotated triple loop

,element,comp_id,skater_fullname,event,bv,goe
30943,3Lo<,GPCAN_1920,Veronik MALLET,LADIES_FS,3.92,-1.96
30944,2S,GPCAN_1920,Veronik MALLET,LADIES_FS,1.30,0.06
30945,3T,GPCAN_1920,Veronik MALLET,LADIES_FS,4.62,0.24
30946,2A+2T,GPCAN_1920,Veronik MALLET,LADIES_FS,5.06,0.52
30947,2A,GPCAN_1920,Veronik MALLET,LADIES_FS,3.63,0.38


In [0]:
error_data = {
    "edge_call" : [],
    "underrotated" : [],
    "downgraded" : [],
    "invalid" : [],
    "other_error" : []
}

# Key phrases that indicate an error
errors = ["+COMBO", "+REP", "!", "*", "e", "<<", "<"]

# Clean up jump elements and record errors
def clean_jumps(element):
  bad_edge = 0
  is_under = 0
  invalid = 0
  is_dg = 0
  has_other_err = 0

  for error in errors:
    if error in element:
      if is_other_error(error):
        has_other_err = 1
      elif error == "e":
        bad_edge = 1
      elif error == "*":
        invalid = 1
      elif error == "<<":
        is_dg = 1
      else:
        is_under = 1
      element = element.replace(error, "")
  
  # Create column data based on error data
  error_data["edge_call"].append(bad_edge)
  error_data["underrotated"].append(is_under)
  error_data["downgraded"].append(is_dg)
  error_data["invalid"].append(invalid)
  error_data["other_error"].append(has_other_err)
  return element

# Determines if a value is of the other_error category
def is_other_error(value):
  return ((value == "+COMBO") or 
          (value == "+REP") or
          (value == "!"))

In [40]:
# Clean jump element data
df_jumps["element"] = (df_jumps["element"].apply(clean_jumps))

# Add jump error data to original DataFrame
for key in error_data.keys():
  df_jumps[key] = error_data[key]

df_jumps

,element,comp_id,skater_fullname,event,bv,goe,edge_call,underrotated,downgraded,invalid,other_error
0,4T+3T,WC_1011,Patrick CHAN,MEN_SP,14.40,1.86,0,0,0,0,0
1,3A,WC_1011,Patrick CHAN,MEN_SP,8.50,1.86,0,0,0,0,0
2,3F,WC_1011,Patrick CHAN,MEN_SP,5.30,1.50,0,0,0,0,0
3,4T,WC_1011,Nobunari ODA,MEN_SP,10.30,-2.43,0,0,0,0,0
4,3A,WC_1011,Nobunari ODA,MEN_SP,8.50,2.00,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
30943,3Lo,GPCAN_1920,Veronik MALLET,LADIES_FS,3.92,-1.96,0,1,0,0,0
30944,2S,GPCAN_1920,Veronik MALLET,LADIES_FS,1.30,0.06,0,0,0,0,0
30945,3T,GPCAN_1920,Veronik MALLET,LADIES_FS,4.62,0.24,0,0,0,0,0
30946,2A+2T,GPCAN_1920,Veronik MALLET,LADIES_FS,5.06,0.52,0,0,0,0,0


To summarize error data for each jump element, I summed all errors to a num_jump_errors column. This encompasses all errors for each element in a single column.

I also summed the number of each type of major jump (quadruple, triple, and double rotation jumps) into their own columns as well. This will indicate how many of a certain jump a skater performed.

In [41]:
df_jumps["num_jump_errors"] = (df_jumps["edge_call"] +
                          df_jumps["underrotated"] +
                          df_jumps["downgraded"] +
                          df_jumps["invalid"] +
                          df_jumps["other_error"])

df_jumps["num_quads"] = df_jumps.element.str.count("4")
df_jumps["num_triples"] = df_jumps.element.str.count("3")
df_jumps["num_doubles"] = df_jumps.element.str.count("2")
df_jumps

,element,comp_id,skater_fullname,event,bv,goe,edge_call,underrotated,downgraded,invalid,other_error,num_jump_errors,num_quads,num_triples,num_doubles
0,4T+3T,WC_1011,Patrick CHAN,MEN_SP,14.40,1.86,0,0,0,0,0,0,1,1,0
1,3A,WC_1011,Patrick CHAN,MEN_SP,8.50,1.86,0,0,0,0,0,0,0,1,0
2,3F,WC_1011,Patrick CHAN,MEN_SP,5.30,1.50,0,0,0,0,0,0,0,1,0
3,4T,WC_1011,Nobunari ODA,MEN_SP,10.30,-2.43,0,0,0,0,0,0,1,0,0
4,3A,WC_1011,Nobunari ODA,MEN_SP,8.50,2.00,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30943,3Lo,GPCAN_1920,Veronik MALLET,LADIES_FS,3.92,-1.96,0,1,0,0,0,1,0,1,0
30944,2S,GPCAN_1920,Veronik MALLET,LADIES_FS,1.30,0.06,0,0,0,0,0,0,0,0,1
30945,3T,GPCAN_1920,Veronik MALLET,LADIES_FS,4.62,0.24,0,0,0,0,0,0,0,1,0
30946,2A+2T,GPCAN_1920,Veronik MALLET,LADIES_FS,5.06,0.52,0,0,0,0,0,0,0,0,2


Similarly, spins that fail to meet the spin requirements are indicated by "V" and invalid spins are indicated with "*". I also strip these characters out so the same spin elements can be compared to each other, and summed number errors to summarize error data for each spin.

In [0]:
failed_req = []
invalid_spin = []

# Clean up spins elements and record errors
def clean_spins(element):
  error = 0
  invalid = 0
  if "V" in element:
    element = element.replace("V", "")
    error = 1
  if "*" in element:
    element = element.replace("*", "")
    invalid = 1
    
  failed_req.append(error)
  invalid_spin.append(invalid)
  return element

In [43]:
# Clean spin element data
df_spins["element"] = (df_spins["element"].apply(clean_spins))

# Add spin error data to original DataFrame
df_spins["failed_req"] = failed_req
df_spins["invalid"] = invalid_spin

# Sum errors
df_spins["num_spin_errors"] = (df_spins["failed_req"] +
                          df_spins["invalid"])

df_spins

,element,comp_id,skater_fullname,event,bv,goe,failed_req,invalid,num_spin_errors
0,CCoSp4,WC_1011,Patrick CHAN,MEN_SP,3.50,0.93,0,0,0
1,FSSp4,WC_1011,Patrick CHAN,MEN_SP,3.00,1.00,0,0,0
2,CCSp3,WC_1011,Patrick CHAN,MEN_SP,2.80,0.64,0,0,0
3,FCSp4,WC_1011,Nobunari ODA,MEN_SP,3.20,0.36,0,0,0
4,CCoSp2,WC_1011,Nobunari ODA,MEN_SP,2.50,0.86,0,0,0
...,...,...,...,...,...,...,...,...,...
18456,FCCoSp3,GPCAN_1920,Gabrielle DALEMAN,LADIES_FS,3.00,0.51,0,0,0
18457,CCoSp4,GPCAN_1920,Gabrielle DALEMAN,LADIES_FS,3.50,0.40,0,0,0
18458,FCCoSp4,GPCAN_1920,Veronik MALLET,LADIES_FS,3.50,0.65,0,0,0
18459,FSSp4,GPCAN_1920,Veronik MALLET,LADIES_FS,3.00,0.64,0,0,0


Finally, I associatied the date of the competition to each element, then saving elements data into their own .csv files.

In [0]:
df_comps = df_comps.drop(columns="link") # drop link column since it's no longer needed
df_jumps.merge(df_comps, on="comp_id").to_csv(path_or_buf=data_dir + "jumps_data.csv", index=False)
df_spins.merge(df_comps, on="comp_id").to_csv(path_or_buf=data_dir + "spins_data.csv", index=False)
df_pcs.merge(df_comps, on="comp_id").to_csv(path_or_buf=data_dir + "pcs_data.csv", index=False)

### Merging judge panels with their nationalities

I associate the nationalities of every judge in the judge panel by merging df_judge_panels with df_judges.



In [46]:
for i in range(1, 10):
  judge = "j%d" % i
  df_judge_panels = (df_judge_panels
                     .merge(df_judges, left_on=judge + "_name", right_on="judge_fullname")
                     .rename(columns={"judge_nat" : judge + "_nat"})
                     .drop(columns="judge_fullname"))

df_judge_panels

,comp_id,event,j1_name,j2_name,j3_name,j4_name,j5_name,j6_name,j7_name,j8_name,j9_name,j1_nat,j2_nat,j3_nat,j4_nat,j5_nat,j6_nat,j7_nat,j8_nat,j9_nat
0,WC_1011,MEN_SP,Lovorka KODRIN,Senem AHISKAL,Beatrice PFISTER,Paula M NAUGHTON,Diana STEVENS,Daniel DELFA,Seppo KURTTI,Maria MILLER,Julia ANDREEVA,CRO,TUR,SUI,USA,GBR,ESP,ISU,ISU,RUS
1,GPF_1415,MEN_FS,Joseph INMAN,Yukiko OKABE,Jarmila PORTOVA,Allan BOEHM,Marta OLOZAGARRE,Jung-Sue LEE,Susan HEFFERNAN,Yumin WANG,Julia ANDREEVA,USA,JPN,CZE,SVK,ESP,KOR,CAN,CHN,RUS
2,GPCHN_1011,LADIES_SP,Janice HUNTER,Robert ROSENBLUTH,Nobuhiko YOSHIOKA,Evgeni ROKHIN,Philippe MERIGUET,Hailan JIANG,Peter LEVIN,You-Hwa LEE,Julia ANDREEVA,CAN,USA,JPN,UZB,FRA,CHN,SWE,KOR,RUS
3,GPFRA_1617,MEN_FS,Francoise DE RAPPARD,Masako KUBOTA,Yuri GUSKOV,Saodat NUMANOVA,Elisabeth LOUESDON,Daniel DELFA,Massimo ORLANDINI,Olga KOZHEMYAKINA,Steve WINKLER,BEL,JPN,KAZ,UZB,FRA,ESP,ITA,RUS,USA
4,GPFRA_1920,MEN_SP,Salome CHIGOGIDZE,Deveny DECK,Natasa SKOK,Janice HUNTER,Kerstin KIMMINUS,Philippe MERIGUET,Alexei BELETSKI,Olga KOZHEMYAKINA,Nobuhiko YOSHIOKA,GEO,USA,SLO,CAN,GER,FRA,ISR,RUS,JPN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,EC_1415,LADIES_FS,Ulla FAIG,Philippe MERIGUET,Anu NIINIRANTA,Christiane MILES,Lone AMSTRUP LINNEMANN,Zanna KULIK,Alla SHEKHOVTSOVA,Osman SIRVAN,Natalia KRUGLOVA,GER,FRA,FIN,SUI,DEN,EST,RUS,TUR,UKR
398,WC_1112,MEN_SP,Evgeni ROKHIN,Eddy WU,Sung-Hee KOH,Anna KANTOR,Pirjo ELO,Ekaterina SEROVA,Ebru ANILDI,Inger ANDERSSON,Katalin BALCZO,UZB,TWN,KOR,ISR,FIN,BLR,TUR,SWE,HUN
399,GPRUS_1819,LADIES_SP,Julia ANDREEVA,Deborah CURRIE,Miwako ANDO,Attila SOOS,Anja RIST,Yuriy GUSKOV,Malgorzata SOBKOW,Kyung Won SAKONG,Claudia FASSORA,RUS,USA,JPN,HUN,GER,KAZ,POL,KOR,SUI
400,GPRUS_1819,LADIES_FS,Julia ANDREEVA,Deborah CURRIE,Miwako ANDO,Attila SOOS,Anja RIST,Yuriy GUSKOV,Malgorzata SOBKOW,Kyung Won SAKONG,Claudia FASSORA,RUS,USA,JPN,HUN,GER,KAZ,POL,KOR,SUI


Now saving judge panel data, just in case.

In [0]:
df_judge_panels.to_csv(path_or_buf=data_dir + "judge_panels_data.csv", index=False)

### Cleaning up and unifying all skater data

First, I combine short program and long program data for each gender. I add a gender column for both men and ladies dataframes, then concatenate all skater data into a single skater dataframe.

In [48]:
df_men_skate = pd.concat([df_men_sp, df_men_fs], ignore_index=True)
df_men_skate["gender"] = 'M'
df_men_skate

,comp_id,event,skater_fullname,skater_nat,comp_rank,total_score,ded,bv,tes,pcs,j1_tes,j2_tes,j3_tes,j4_tes,j5_tes,j6_tes,j7_tes,j8_tes,j9_tes,j1_pcs,j2_pcs,j3_pcs,j4_pcs,j5_pcs,j6_pcs,j7_pcs,j8_pcs,j9_pcs,gender
0,WC_1011,MEN_SP,Patrick CHAN,CAN,1,93.02,0.00,41.40,51.48,41.54,49.80,52.00,52.30,52.80,49.80,51.30,50.80,52.00,53.30,38.75,43.25,45.00,47.50,39.25,39.50,40.50,39.25,43.00,M
1,WC_1011,MEN_SP,Nobunari ODA,JPN,2,81.81,0.00,40.90,43.37,38.44,42.60,43.10,45.30,41.20,42.70,44.30,44.30,43.80,41.80,37.50,41.00,40.00,37.50,37.75,39.25,36.50,35.50,41.25,M
2,WC_1011,MEN_SP,Daisuke TAKAHASHI,JPN,3,80.25,0.00,35.40,39.33,40.92,38.90,38.80,39.80,40.10,42.00,40.80,38.40,38.10,36.50,40.75,40.75,40.75,41.00,41.75,37.75,40.50,41.25,44.50,M
3,WC_1011,MEN_SP,Artur GACHINSKI,RUS,4,78.34,0.00,39.60,42.45,35.89,40.40,43.20,43.70,40.20,42.70,43.40,42.70,43.20,43.20,35.25,36.00,40.50,32.50,37.00,36.50,35.75,34.75,35.50,M
4,WC_1011,MEN_SP,Florent AMODIO,FRA,5,77.64,0.00,35.50,40.46,37.18,38.50,41.50,41.40,39.50,41.00,40.00,40.00,40.70,40.30,34.25,38.75,37.25,34.00,38.00,36.25,38.25,41.50,36.00,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061,GPCAN_1920,MEN_FS,Roman SADOVSKY,CAN,8,139.06,-2.00,69.47,63.78,77.28,8 65.82,10 61.72,7 66.26,9 61.63,8 65.55,10 63.10,10 62.25,9 63.26,9 64.90,8 77.50,8 73.00,5 86.50,10 76.00,5 80.50,9 75.50,10 73.50,6 78.50,7 78.50,M
3062,GPCAN_1920,MEN_FS,Brendan KERRY,AUS,9,137.02,0.00,63.95,66.46,70.56,7 67.57,7 65.26,8 64.57,7 65.60,7 67.68,7 67.37,7 67.11,8 64.78,7 68.57,12 69.00,12 68.00,10 76.00,12 67.50,11 71.00,10 74.00,11 72.50,12 67.50,12 71.50,M
3063,GPCAN_1920,MEN_FS,Julian Zhi Jie YEE,MAS,10,135.99,0.00,57.63,62.21,73.78,11 61.18,11 60.39,11 60.31,11 59.53,11 64.12,9 63.85,9 64.84,7 65.15,11 61.20,9 73.50,10 69.00,12 72.00,11 73.50,9 78.00,10 74.00,9 77.50,9 74.00,10 72.00,M
3064,GPCAN_1920,MEN_FS,Paul FENTZ,GER,11,135.92,0.00,62.05,63.40,72.52,9 64.17,9 63.20,9 62.71,8 63.16,10 64.15,11 62.93,8 65.27,11 61.41,10 63.05,11 72.50,9 72.00,11 74.00,9 76.50,12 70.50,12 71.50,12 72.00,11 73.00,10 72.00,M


In [49]:
df_ladies_skate = pd.concat([df_ladies_sp, df_ladies_fs], ignore_index=True)
df_ladies_skate["gender"] = 'F'
df_ladies_skate

,comp_id,event,skater_fullname,skater_nat,comp_rank,total_score,ded,bv,tes,pcs,j1_tes,j2_tes,j3_tes,j4_tes,j5_tes,j6_tes,j7_tes,j8_tes,j9_tes,j1_pcs,j2_pcs,j3_pcs,j4_pcs,j5_pcs,j6_pcs,j7_pcs,j8_pcs,j9_pcs,gender
0,WC_1011,LADIES_SP,Yuna KIM,KOR,1,65.91,0.00,28.50,32.97,32.94,35.00,32.10,33.80,33.30,35.20,32.30,32.80,33.70,31.10,33.40,33.00,33.00,34.80,36.20,31.40,32.80,31.40,31.60,F
1,WC_1011,LADIES_SP,Miki ANDO,JPN,2,65.58,0.00,28.70,34.20,31.38,33.80,33.80,35.50,35.50,33.60,33.10,34.80,34.30,33.10,31.00,32.20,32.60,31.80,30.60,31.40,30.60,31.60,30.60,F
2,WC_1011,LADIES_SP,Ksenia MAKAROVA,RUS,3,61.62,0.00,28.80,32.53,29.09,33.20,33.20,32.70,31.00,32.70,31.50,33.90,32.00,31.70,30.40,30.20,28.80,28.60,28.60,30.40,29.00,27.20,28.20,F
3,WC_1011,LADIES_SP,Alissa CZISNY,USA,4,61.47,0.00,27.70,31.33,30.14,31.20,31.90,32.50,33.20,31.50,30.30,29.30,31.20,31.30,29.40,30.60,32.00,31.60,31.40,29.00,27.60,30.20,29.20,F
4,WC_1011,LADIES_SP,Alena LEONOVA,RUS,5,59.75,0.00,28.30,31.55,28.20,31.90,32.40,30.20,30.70,32.20,32.40,30.20,32.20,31.70,27.60,30.80,25.00,23.60,29.00,30.60,27.80,28.40,29.20,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3083,GPCAN_1920,LADIES_FS,Serafima SAKHANOVICH,RUS,8,113.34,0.00,48.91,53.98,59.36,7 54.89,7 55.26,8 51.87,8 53.21,8 52.36,7 56.57,8 50.66,8 54.86,8 53.19,7 60.80,6 62.00,8 58.80,9 58.80,8 58.00,8 59.60,8 58.40,8 60.40,9 58.40,F
3084,GPCAN_1920,LADIES_FS,Alexia PAGANINI,SUI,9,105.52,-1.00,52.50,50.70,55.82,9 50.35,10 51.22,10 49.19,9 51.96,9 50.95,9 53.05,9 50.19,10 50.52,10 50.18,10 54.80,11 55.60,10 52.80,10 55.60,9 56.40,9 58.40,9 56.40,9 55.60,10 56.40,F
3085,GPCAN_1920,LADIES_FS,Alicia PINEAULT,CAN,10,103.78,0.00,49.91,50.76,53.02,10 49.96,9 52.21,9 50.48,10 50.45,10 49.86,10 52.23,10 48.43,9 51.10,9 51.05,12 52.40,10 56.40,12 50.80,11 54.00,12 52.40,11 52.40,12 49.60,11 53.60,11 56.00,F
3086,GPCAN_1920,LADIES_FS,Gabrielle DALEMAN,CAN,11,100.40,-4.00,50.33,46.34,58.06,11 45.93,11 48.76,11 46.00,11 48.40,11 45.93,12 45.40,12 43.72,11 46.14,11 45.71,9 59.20,8 60.40,8 58.80,6 61.20,10 55.60,10 56.00,10 55.60,10 54.80,8 60.40,F


In [50]:
df_skaters = pd.concat([df_ladies_skate, df_men_skate])
df_skaters.reset_index(inplace=True, drop=True)
df_skaters

,comp_id,event,skater_fullname,skater_nat,comp_rank,total_score,ded,bv,tes,pcs,j1_tes,j2_tes,j3_tes,j4_tes,j5_tes,j6_tes,j7_tes,j8_tes,j9_tes,j1_pcs,j2_pcs,j3_pcs,j4_pcs,j5_pcs,j6_pcs,j7_pcs,j8_pcs,j9_pcs,gender
0,WC_1011,LADIES_SP,Yuna KIM,KOR,1,65.91,0.00,28.50,32.97,32.94,35.00,32.10,33.80,33.30,35.20,32.30,32.80,33.70,31.10,33.40,33.00,33.00,34.80,36.20,31.40,32.80,31.40,31.60,F
1,WC_1011,LADIES_SP,Miki ANDO,JPN,2,65.58,0.00,28.70,34.20,31.38,33.80,33.80,35.50,35.50,33.60,33.10,34.80,34.30,33.10,31.00,32.20,32.60,31.80,30.60,31.40,30.60,31.60,30.60,F
2,WC_1011,LADIES_SP,Ksenia MAKAROVA,RUS,3,61.62,0.00,28.80,32.53,29.09,33.20,33.20,32.70,31.00,32.70,31.50,33.90,32.00,31.70,30.40,30.20,28.80,28.60,28.60,30.40,29.00,27.20,28.20,F
3,WC_1011,LADIES_SP,Alissa CZISNY,USA,4,61.47,0.00,27.70,31.33,30.14,31.20,31.90,32.50,33.20,31.50,30.30,29.30,31.20,31.30,29.40,30.60,32.00,31.60,31.40,29.00,27.60,30.20,29.20,F
4,WC_1011,LADIES_SP,Alena LEONOVA,RUS,5,59.75,0.00,28.30,31.55,28.20,31.90,32.40,30.20,30.70,32.20,32.40,30.20,32.20,31.70,27.60,30.80,25.00,23.60,29.00,30.60,27.80,28.40,29.20,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149,GPCAN_1920,MEN_FS,Roman SADOVSKY,CAN,8,139.06,-2.00,69.47,63.78,77.28,8 65.82,10 61.72,7 66.26,9 61.63,8 65.55,10 63.10,10 62.25,9 63.26,9 64.90,8 77.50,8 73.00,5 86.50,10 76.00,5 80.50,9 75.50,10 73.50,6 78.50,7 78.50,M
6150,GPCAN_1920,MEN_FS,Brendan KERRY,AUS,9,137.02,0.00,63.95,66.46,70.56,7 67.57,7 65.26,8 64.57,7 65.60,7 67.68,7 67.37,7 67.11,8 64.78,7 68.57,12 69.00,12 68.00,10 76.00,12 67.50,11 71.00,10 74.00,11 72.50,12 67.50,12 71.50,M
6151,GPCAN_1920,MEN_FS,Julian Zhi Jie YEE,MAS,10,135.99,0.00,57.63,62.21,73.78,11 61.18,11 60.39,11 60.31,11 59.53,11 64.12,9 63.85,9 64.84,7 65.15,11 61.20,9 73.50,10 69.00,12 72.00,11 73.50,9 78.00,10 74.00,9 77.50,9 74.00,10 72.00,M
6152,GPCAN_1920,MEN_FS,Paul FENTZ,GER,11,135.92,0.00,62.05,63.40,72.52,9 64.17,9 63.20,9 62.71,8 63.16,10 64.15,11 62.93,8 65.27,11 61.41,10 63.05,11 72.50,9 72.00,11 74.00,9 76.50,12 70.50,12 71.50,12 72.00,11 73.00,10 72.00,M


It appears that some observations contain judge score data that have leading numbers in it (most prevalent in more recent data). I strip those leading values so that they can be properly read in as numeric values.

Some observations have empty judge score values (for example, there are some events where there were only 8 judges instead of 9). I fill those NaN values with 0, since it's not possible for a judge to give a TES or PCS score of 0 to a skater (unless they somehow manage to invalidate **EVERY** single element they performed).

In [0]:
# Clean up judge score values
def clean_scores(value):
  return value.split(' ', 1)[-1]

# Fill empty values with 0
df_skaters.fillna(value="0", inplace=True)

# Clean up judge scores for both TES and PCS
for i in range(1, 10):
  column_tes = "j%d_tes" % i
  column_pcs = "j%d_pcs" % i

  df_skaters[column_tes] = (df_skaters[column_tes]
                               .apply(clean_scores))
  df_skaters[column_pcs] = (df_skaters[column_pcs]
                               .apply(clean_scores))

Then, I associate each judge panel to the skater by performing another merge with df_judge_panels.

In [52]:
df_skaters = df_skaters.merge(df_judge_panels, on=["comp_id", "event"])
df_skaters

,comp_id,event,skater_fullname,skater_nat,comp_rank,total_score,ded,bv,tes,pcs,j1_tes,j2_tes,j3_tes,j4_tes,j5_tes,j6_tes,j7_tes,j8_tes,j9_tes,j1_pcs,j2_pcs,j3_pcs,j4_pcs,j5_pcs,j6_pcs,j7_pcs,j8_pcs,j9_pcs,gender,j1_name,j2_name,j3_name,j4_name,j5_name,j6_name,j7_name,j8_name,j9_name,j1_nat,j2_nat,j3_nat,j4_nat,j5_nat,j6_nat,j7_nat,j8_nat,j9_nat
0,WC_1011,LADIES_SP,Yuna KIM,KOR,1,65.91,0.00,28.50,32.97,32.94,35.00,32.10,33.80,33.30,35.20,32.30,32.80,33.70,31.10,33.40,33.00,33.00,34.80,36.20,31.40,32.80,31.40,31.60,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS
1,WC_1011,LADIES_SP,Miki ANDO,JPN,2,65.58,0.00,28.70,34.20,31.38,33.80,33.80,35.50,35.50,33.60,33.10,34.80,34.30,33.10,31.00,32.20,32.60,31.80,30.60,31.40,30.60,31.60,30.60,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS
2,WC_1011,LADIES_SP,Ksenia MAKAROVA,RUS,3,61.62,0.00,28.80,32.53,29.09,33.20,33.20,32.70,31.00,32.70,31.50,33.90,32.00,31.70,30.40,30.20,28.80,28.60,28.60,30.40,29.00,27.20,28.20,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS
3,WC_1011,LADIES_SP,Alissa CZISNY,USA,4,61.47,0.00,27.70,31.33,30.14,31.20,31.90,32.50,33.20,31.50,30.30,29.30,31.20,31.30,29.40,30.60,32.00,31.60,31.40,29.00,27.60,30.20,29.20,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS
4,WC_1011,LADIES_SP,Alena LEONOVA,RUS,5,59.75,0.00,28.30,31.55,28.20,31.90,32.40,30.20,30.70,32.20,32.40,30.20,32.20,31.70,27.60,30.80,25.00,23.60,29.00,30.60,27.80,28.40,29.20,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149,GPCAN_1920,MEN_FS,Roman SADOVSKY,CAN,8,139.06,-2.00,69.47,63.78,77.28,65.82,61.72,66.26,61.63,65.55,63.10,62.25,63.26,64.90,77.50,73.00,86.50,76.00,80.50,75.50,73.50,78.50,78.50,M,Lynne DEY,Raffaella LOCATELLI,David MUNOZ,Lisa JELINEK,Stefanie SCHMID MATHEWSON,Maria GRIBONOSOVA-GREBNEVA,Anja RIST,Sakae YAMAMOTO,Agita ABELE,CAN,ITA,ESP,AUS,USA,RUS,GER,JPN,LAT
6150,GPCAN_1920,MEN_FS,Brendan KERRY,AUS,9,137.02,0.00,63.95,66.46,70.56,67.57,65.26,64.57,65.60,67.68,67.37,67.11,64.78,68.57,69.00,68.00,76.00,67.50,71.00,74.00,72.50,67.50,71.50,M,Lynne DEY,Raffaella LOCATELLI,David MUNOZ,Lisa JELINEK,Stefanie SCHMID MATHEWSON,Maria GRIBONOSOVA-GREBNEVA,Anja RIST,Sakae YAMAMOTO,Agita ABELE,CAN,ITA,ESP,AUS,USA,RUS,GER,JPN,LAT
6151,GPCAN_1920,MEN_FS,Julian Zhi Jie YEE,MAS,10,135.99,0.00,57.63,62.21,73.78,61.18,60.39,60.31,59.53,64.12,63.85,64.84,65.15,61.20,73.50,69.00,72.00,73.50,78.00,74.00,77.50,74.00,72.00,M,Lynne DEY,Raffaella LOCATELLI,David MUNOZ,Lisa JELINEK,Stefanie SCHMID MATHEWSON,Maria GRIBONOSOVA-GREBNEVA,Anja RIST,Sakae YAMAMOTO,Agita ABELE,CAN,ITA,ESP,AUS,USA,RUS,GER,JPN,LAT
6152,GPCAN_1920,MEN_FS,Paul FENTZ,GER,11,135.92,0.00,62.05,63.40,72.52,64.17,63.20,62.71,63.16,64.15,62.93,65.27,61.41,63.05,72.50,72.00,74.00,76.50,70.50,71.50,72.00,73.00,72.00,M,Lynne DEY,Raffaella LOCATELLI,David MUNOZ,Lisa JELINEK,Stefanie SCHMID MATHEWSON,Maria GRIBONOSOVA-GREBNEVA,Anja RIST,Sakae YAMAMOTO,Agita ABELE,CAN,ITA,ESP,AUS,USA,RUS,GER,JPN,LAT


Next, I merge element summarizations into the skater DataFrame. I also associate the date of each skater's competition by merging it with df_comps.

In [53]:
main_cols = ["comp_id", "skater_fullname", "event"]
jumps_groupby = df_jumps.groupby(main_cols)
spins_groupby = df_spins.groupby(main_cols)

for col in ["num_quads", "num_triples", "num_doubles", "num_jump_errors"]:
  sum_attr = jumps_groupby[col].sum()
  df_sum = sum_attr.to_frame().reset_index()
  df_skaters = df_skaters.merge(df_sum, on=main_cols)

num_spin_errors = df_spins.groupby(["comp_id", "skater_fullname", "event"])["num_spin_errors"].sum()
df_spin_err = num_spin_errors.to_frame().reset_index()
df_skaters = df_skaters.merge(df_spin_err, on=main_cols)

# Merge to get date
df_skaters = df_skaters.merge(df_comps, on="comp_id")
df_skaters

,comp_id,event,skater_fullname,skater_nat,comp_rank,total_score,ded,bv,tes,pcs,j1_tes,j2_tes,j3_tes,j4_tes,j5_tes,j6_tes,j7_tes,j8_tes,j9_tes,j1_pcs,j2_pcs,j3_pcs,j4_pcs,j5_pcs,j6_pcs,j7_pcs,j8_pcs,j9_pcs,gender,j1_name,j2_name,j3_name,j4_name,j5_name,j6_name,j7_name,j8_name,j9_name,j1_nat,j2_nat,j3_nat,j4_nat,j5_nat,j6_nat,j7_nat,j8_nat,j9_nat,num_quads,num_triples,num_doubles,num_jump_errors,num_spin_errors,date
0,WC_1011,LADIES_SP,Yuna KIM,KOR,1,65.91,0.00,28.50,32.97,32.94,35.00,32.10,33.80,33.30,35.20,32.30,32.80,33.70,31.10,33.40,33.00,33.00,34.80,36.20,31.40,32.80,31.40,31.60,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS,0,2,2,0,0,2011-04-25
1,WC_1011,LADIES_SP,Miki ANDO,JPN,2,65.58,0.00,28.70,34.20,31.38,33.80,33.80,35.50,35.50,33.60,33.10,34.80,34.30,33.10,31.00,32.20,32.60,31.80,30.60,31.40,30.60,31.60,30.60,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS,0,2,2,0,0,2011-04-25
2,WC_1011,LADIES_SP,Ksenia MAKAROVA,RUS,3,61.62,0.00,28.80,32.53,29.09,33.20,33.20,32.70,31.00,32.70,31.50,33.90,32.00,31.70,30.40,30.20,28.80,28.60,28.60,30.40,29.00,27.20,28.20,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS,0,3,1,0,0,2011-04-25
3,WC_1011,LADIES_SP,Alissa CZISNY,USA,4,61.47,0.00,27.70,31.33,30.14,31.20,31.90,32.50,33.20,31.50,30.30,29.30,31.20,31.30,29.40,30.60,32.00,31.60,31.40,29.00,27.60,30.20,29.20,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS,0,2,2,1,0,2011-04-25
4,WC_1011,LADIES_SP,Alena LEONOVA,RUS,5,59.75,0.00,28.30,31.55,28.20,31.90,32.40,30.20,30.70,32.20,32.40,30.20,32.20,31.70,27.60,30.80,25.00,23.60,29.00,30.60,27.80,28.40,29.20,F,Prisca BINZ-MOSER,Igor FEDCHENKO,Saioa SANCHO,Georgi DIMKOV,Evgeni ROKHIN,Tarja RISTANEN,Stanislava SMIDOVA,Igor DOLGUSHIN,Deborah NOYES,SUI,UKR,ESP,ISU,UZB,FIN,CZE,RUS,AUS,0,3,1,0,0,2011-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149,GPCAN_1920,MEN_FS,Roman SADOVSKY,CAN,8,139.06,-2.00,69.47,63.78,77.28,65.82,61.72,66.26,61.63,65.55,63.10,62.25,63.26,64.90,77.50,73.00,86.50,76.00,80.50,75.50,73.50,78.50,78.50,M,Lynne DEY,Raffaella LOCATELLI,David MUNOZ,Lisa JELINEK,Stefanie SCHMID MATHEWSON,Maria GRIBONOSOVA-GREBNEVA,Anja RIST,Sakae YAMAMOTO,Agita ABELE,CAN,ITA,ESP,AUS,USA,RUS,GER,JPN,LAT,2,6,0,4,0,2019-10-25
6150,GPCAN_1920,MEN_FS,Brendan KERRY,AUS,9,137.02,0.00,63.95,66.46,70.56,67.57,65.26,64.57,65.60,67.68,67.37,67.11,64.78,68.57,69.00,68.00,76.00,67.50,71.00,74.00,72.50,67.50,71.50,M,Lynne DEY,Raffaella LOCATELLI,David MUNOZ,Lisa JELINEK,Stefanie SCHMID MATHEWSON,Maria GRIBONOSOVA-GREBNEVA,Anja RIST,Sakae YAMAMOTO,Agita ABELE,CAN,ITA,ESP,AUS,USA,RUS,GER,JPN,LAT,1,5,3,0,0,2019-10-25
6151,GPCAN_1920,MEN_FS,Julian Zhi Jie YEE,MAS,10,135.99,0.00,57.63,62.21,73.78,61.18,60.39,60.31,59.53,64.12,63.85,64.84,65.15,61.20,73.50,69.00,72.00,73.50,78.00,74.00,77.50,74.00,72.00,M,Lynne DEY,Raffaella LOCATELLI,David MUNOZ,Lisa JELINEK,Stefanie SCHMID MATHEWSON,Maria GRIBONOSOVA-GREBNEVA,Anja RIST,Sakae YAMAMOTO,Agita ABELE,CAN,ITA,ESP,AUS,USA,RUS,GER,JPN,LAT,0,6,4,1,0,2019-10-25
6152,GPCAN_1920,MEN_FS,Paul FENTZ,GER,11,135.92,0.00,62.05,63.40,72.52,64.17,63.20,62.71,63.16,64.15,62.93,65.27,61.41,63.05,72.50,72.00,74.00,76.50,70.50,71.50,72.00,73.00,72.00,M,Lynne DEY,Raffaella LOCATELLI,David MUNOZ,Lisa JELINEK,Stefanie SCHMID MATHEWSON,Maria GRIBONOSOVA-GREBNEVA,Anja RIST,Sakae YAMAMOTO,Agita ABELE,CAN,ITA,ESP,AUS,USA

Finally, I save the skater DataFrame as a .csv.

In [0]:
df_skaters.to_csv(path_or_buf=data_dir + "skaters_data.csv", index=False)

# Prepping data for machine learning

Since I plan to be predicting individual judges' scores for skaters, it would make sense to rearrange the data so that a single observation contains a single judge's scoring data. This would effectively multiply the size of the DataFrame by 9.

However, judges' scores where only tied to their identities (name / nationality) beginning in the **2016-2017 season**. Thus, it would only make sense to include data for these years for the machine learning model. If all data was included, it would be inaccurate for any season before 2016-2017.

In [60]:
df_skaters = pd.read_csv(data_dir + "skaters_data.csv")
known_judge_nat = df_skaters[df_skaters.comp_id.str.contains("1617") |
                             df_skaters.comp_id.str.contains("1718") |
                             df_skaters.comp_id.str.contains("1819") |
                             df_skaters.comp_id.str.contains("1920")]
known_judge_nat.reset_index(inplace=True, drop=True)

cols = ["comp_id",
        "skater_fullname", 
        "skater_nat", 
        "gender",
        "bv",
        "num_quads",
        "num_triples",
        "num_doubles",
        "num_jump_errors",
        "num_spin_errors"]
df_skaters_ml = (known_judge_nat[cols + ["j1_nat", "j1_tes", "j1_pcs"]].copy()
      .rename(columns={"j1_nat" : "judge_nat", 
                       "j1_tes" : "judge_tes", 
                       "j1_pcs" : "judge_pcs"}))

for i in range(2, 10):
  j = "j%d" % i
  skater_judge = (known_judge_nat[cols + [j + "_nat", j + "_tes", j + "_pcs"]]
                  .copy()
                  .rename(columns={j + "_nat" : "judge_nat", 
                                   j + "_tes" : "judge_tes", 
                                   j + "_pcs" : "judge_pcs"}))
  df_skaters_ml = pd.concat([df_skaters_ml, skater_judge])

# Remove judge data with zero as scores
df_skaters_ml = df_skaters_ml.loc[df_skaters_ml["judge_tes"] > 0]
df_skaters_ml.reset_index(drop=True, inplace=True)

# Create judge total scores columns
df_skaters_ml["judge_total"] = df_skaters_ml["judge_tes"] + df_skaters_ml["judge_pcs"]
df_skaters_ml

,comp_id,skater_fullname,skater_nat,gender,bv,num_quads,num_triples,num_doubles,num_jump_errors,num_spin_errors,judge_nat,judge_tes,judge_pcs,judge_total
0,WC_1617,Evgenia MEDVEDEVA,RUS,F,33.10,0,3,1,0,0,FRA,40.30,37.0,77.30
1,WC_1617,Kaetlyn OSMOND,CAN,F,32.53,0,3,1,0,0,FRA,39.53,32.8,72.33
2,WC_1617,Gabrielle DALEMAN,CAN,F,31.83,0,3,1,0,0,FRA,38.33,32.2,70.53
3,WC_1617,Anna POGORILAYA,RUS,F,32.84,0,3,1,0,0,FRA,36.24,33.8,70.04
4,WC_1617,Karen CHEN,USA,F,32.24,0,3,1,0,0,FRA,38.84,30.4,69.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22696,GPCAN_1920,Roman SADOVSKY,CAN,M,69.47,2,6,0,4,0,LAT,64.90,78.5,143.40
22697,GPCAN_1920,Brendan KERRY,AUS,M,63.95,1,5,3,0,0,LAT,68.57,71.5,140.07
22698,GPCAN_1920,Julian Zhi Jie YEE,MAS,M,57.63,0,6,4,1,0,LAT,61.20,72.0,133.20
22699,GPCAN_1920,Paul FENTZ,GER,M,62.05,0,7,3,1,0,LAT,63.05,72.0,135.05


Save ML data to .csv

In [0]:
df_skaters_ml.to_csv(path_or_buf=data_dir + "skaters_data_ml.csv", index=False)